In [ ]:
%load_ext autoreload
%autoreload 2

# Generate common non-negatives

Add existing standard, triplets, and nicknames to common non-negatives

In [10]:
import random
import re

import pandas as pd

from nama.data.filesystem import download_file_from_s3, save_file
from nama.data.utils import read_csv

In [1]:
#config
# TODO do for given and surname
given_surname = "given"
# given_surname = "surname"

num_common_names = 10000 if given_surname == "given" else 25000
tree_name_min_freq = 1000

pref_path = f"s3://fs-nama-data/2024/familysearch-names/processed/tree-preferred-{given_surname}-aggr.csv.gz"
std_path = f"../references/std_{given_surname}.txt"
given_nicknames_path = "../references/givenname_nicknames.csv"
triplets_path=f"s3://fs-nama-data/2024/familysearch-names/processed/tree-hr-{given_surname}-triplets-{tree_name_min_freq}.csv.gz"

non_negatives_path = f"s3://fs-nama-data/2024/familysearch-names/processed/common_{given_surname}_non_negatives.csv"

## Load data

### read preferred names

In [3]:
path = download_file_from_s3(pref_path) if pref_path.startswith("s3://") else pref_path
pref_df = read_csv(path)
common_names = set([name for name in pref_df['name'][:num_common_names].tolist() \
                if len(name) > 1 and re.fullmatch(r'[a-z]+', name)])
len(common_names)

9977

## Start with FS buckets

In [4]:
common_names_set = set(common_names)
common_non_negatives = set()

with open(std_path) as f:
    for ix, line in enumerate(f.readlines()):
        line = line.strip()
        head_names, tail_names = line.split(':')
        head_names = head_names.strip()
        tail_names = tail_names.strip()
        names = set()
        if len(head_names):
            names |= set(head_names.split(' '))
        if len(tail_names):
            names |= set(tail_names.split(' '))
        names = [name for name in names if len(name) > 0]
        for name1 in names:
            if name1 not in common_names_set:
                continue
            for name2 in names:
                if name2 not in common_names_set:
                    continue
                if name1 == name2:
                    continue
                common_non_negatives.add((name1, name2))
print(len(common_non_negatives))

163802


### add triplets

In [5]:
path = download_file_from_s3(triplets_path) if triplets_path.startswith("s3://") else triplets_path
triplets_df = read_csv(path)
print(len(triplets_df))
triplets_df.head(3)

3289279


,anchor,positive,positive_score,negative,negative_score
0,aloysius,alaysius,0.961516,aloisia,0.403759
1,rosanna,roseanna,0.792378,susanne,0.400109
2,stephen,stehen,0.968966,stearn,0.501695


In [6]:
for anchor, pos, neg in zip(
    triplets_df['anchor'], 
    triplets_df['positive'],
    triplets_df['negative'],
):
    common_non_negatives.add((anchor, pos))
    common_non_negatives.add((pos, anchor))
    common_non_negatives.add((anchor, neg))
    common_non_negatives.add((neg, anchor))
len(common_non_negatives)

1118810

### add given nicknames

In [7]:
if given_surname == "given":
    with open(given_nicknames_path, "rt") as f:
        for line in f.readlines():
            names = line.split(',')
            for name1 in names:
                for name2 in names:
                    if name1 > name2:
                        common_non_negatives.add((name1, name2))
                        common_non_negatives.add((name2, name1))
len(common_non_negatives)

1122584

## Save common non-negatives

In [11]:
records = []
for name1, name2 in common_non_negatives:
    records.append({'name1': name1, 'name2': name2})
random.shuffle(records)
df = pd.DataFrame(records)
save_file(non_negatives_path,
          lambda local_out_path : df.to_csv(local_out_path, index=False))